# 1 Sequence data
Mainly use a software called `seqkit`.

## 1.1 download
Download reads from NCBI SRA: 
- use `fastq-dump` to split the SRR_ID file: `fastq-dump --gzip --split-3 --defline-qual '+' --defline-seq '@\$ac-\$si/\$ri'   SRR_ID`
- use `prefetch` to download the SRR_ID file: `prefetch -X 200G -O ./ SRR_ID`
```
prefetch -X 200G -O ./ SRR_ID
fasterq-dump --split-3 --defline-seq '@$sn[_$rn]/$ri' --defline-qual '+' SRR_ID
nohup fasterq-dump -p -e 20 --split-3 -O ./ SRR7164628/SRR7164628.sra
```
For instance:
```
#nohup fasterq-dump -e 20 --split-3 -O ./ SRR7164604/SRR7164604.sra && pigz -3 SRR7164604_1.fastq.gz SRR7164604_1.fastq.gz && rm SRR7164604/SRR7164604.sra &
#nohup fasterq-dump -e 20 --split-3 -O ./ SRR7164620/SRR7164620.sra && pigz -3 SRR7164620_1.fastq.gz SRR7164620_1.fastq.gz && rm SRR7164620/SRR7164620.sra &
#nohup fasterq-dump -e 20 --split-3 -O ./ SRR7164628/SRR7164628.sra && pigz -3 SRR7164628_1.fastq SRR7164628_2.fastq && rm SRR7164628/SRR7164628.sra &
#nohup fasterq-dump -e 20 --split-3 -O ./ SRR7164669/SRR7164669.sra && pigz -3 SRR7164669_1.fastq SRR7164669_2.fastq && rm SRR7164669/SRR7164669.sra &
#nohup fasterq-dump -e 20 --split-3 -O ./ SRR7164670/SRR7164670.sra && pigz -3 SRR7164670_1.fastq SRR7164670_2.fastq && rm SRR7164670/SRR7164670.sra &
```

## 1.2 reference genome
move chr1A to certain file 
```
cat iwgsc_refseqv1.0_TransposableElements_2017Mar13.gff3 | grep -w 'chr1A' > chr1A/chr1A.gff3
seqkit grep -p 1 abd_iwgscV1.fa -o chr1A/chr1A1.fa
seqkit grep -f chr1B.name -o chr1B.fa abd_iwgscV1.fa
seqkit stats
```

## 1.3 subsample 
Use a java program I wrote called `ssReads`. Now it can be found in `package pgl.LAW.WeaTE.ssReads` in the `pgl` package.

For instance:
```
java -cp ~/js/pop/src/ ssReads 1 SRR7164604/SRR7164604_1.fastq.gz f1.SRR7164604.fq SRR7164604/SRR7164604_2.fastq.gz r2.SRR7164604.fq 906 && pigz -3 f1.SRR7164604.fq r2.SRR7164604.fq
java -cp ~/js/pop/src/ ssReads 1 SRR7164620/SRR7164620_1.fastq.gz f1.SRR7164620.fq SRR7164620/SRR7164620_2.fastq.gz r2.SRR7164620.fq 878 && pigz -3 f1.SRR7164620.fq r2.SRR7164620.fq
java -cp ~/js/pop/src/ ssReads 1 SRR7164628/SRR7164628_1.fastq.gz f1.SRR7164628.fq SRR7164628/SRR7164628_2.fastq.gz r2.SRR7164628.fq 905 && pigz -3 f1.SRR7164628.fq r2.SRR7164628.fq
java -cp ~/js/pop/src/ ssReads 1 SRR7164669/SRR7164669_1.fastq.gz f1.SRR7164669.fq SRR7164669/SRR7164669_2.fastq.gz r2.SRR7164669.fq 902 && pigz -3 f1.SRR7164669.fq r2.SRR7164669.fq
java -cp ~/js/pop/src/ ssReads 1 SRR7164670/SRR7164670_1.fastq.gz f1.SRR7164670.fq SRR7164670/SRR7164670_2.fastq.gz r2.SRR7164670.fq 917 && pigz -3 f1.SRR7164670.fq r2.SRR7164670.fq
```
Before using this program, you should count the read number based on the sequencing depth. For example, if the sequencing depth is 11.05259133x, and I need 1x reads, then the number of reads should be: `10000/11.05259133 = 904.765`

## 1.4 generate reads
Sometimes, for benchmark, we need to generate reads from the reference genome.

This can be done using a program called 'iss': `nohup iss generate -p 10 -g kg.1.fa -n 2666667  --mode basic -o ./ &`

## 1.5 grep sequence from reference genome
This method can be used on any '.fa' files:
```
awk -F'\t' '{print $1}' vu.fai > name
seqkit grep -f name 1A.lib.fa > 1A.known.fa
```
To use this method, first you should create a `.fai` file. This can be done by: `samtools faidx 1A.lib.fa`.

## 1.6 merge genome sequence
As some programs require the genome length is less than the normal length of a wheat genome, we usually split the genome into two parts. If you want to get the full-length genome sequence data, you can use the method below:
```
line_num=$(grep -n '>32' chr6A.fa | cut -d : -f 1)
sed -i "${line_num}d" chr6A.fa
sed -i 's/>31/>chr6A/g' chr6A.fa
awk '/^>/&&NR>1{print "";}{printf "%s",/^>/?$0"\n":$0}' chr6A.fa > chr6A1.fa
seqkit seq chr6A1.fa > chr6A.fa
rm chr6A1.fa
samtools faidx chr6A.fa
```
Or, you can use a script writen by senior student Jiayu Dong:

In [ ]:
%%bash
# file name: wheat_chr_merge.sh

infile=$1
# merge genome files (formerly split by chr)
cat ${1}| awk 'BEGIN{FS="\t";OFS="\t"}{

        if($1=="2"){$1="1A";$2=$2+471304005;$3=$3+471304005;print $0}
        else if($1==1){$1="1A";print $0}
        else if($1==3){$1="1B";print $0}
        else if($1==5){$1="1D";print $0}
        else if($1==7){$1="2A";print $0}
        else if($1==9){$1="2B";print $0}
        else if($1==11){$1="2D";print $0}
        else if($1==13){$1="3A";print $0}
        else if($1==15){$1="3B";print $0}
        else if($1==17){$1="3D";print $0}
        else if($1==19){$1="4A";print $0}
        else if($1==21){$1="4B";print $0}
        else if($1==23){$1="4D";print $0}
        else if($1==25){$1="5A";print $0}
        else if($1==27){$1="5B";print $0}
        else if($1==29){$1="5D";print $0}
        else if($1==31){$1="6A";print $0}
        else if($1==33){$1="6B";print $0}
        else if($1==35){$1="6D";print $0}
        else if($1==37){$1="7A";print $0}
        else if($1==39){$1="7B";print $0}
        else if($1==41){$1="7D";print $0}
        else if($1=="4"){$1="1B";$2=$2+438720154;$3=$3+438720154;print $0}
        else if($1=="6"){$1="1D";$2=$2+452179604;$3=$3+452179604;print $0}
        else if($1=="8"){$1="2A";$2=$2+462376173;$3=$3+462376173;print $0}
        else if($1=="10"){$1="2B";$2=$2+453218924;$3=$3+453218924;print $0}
        else if($1=="12"){$1="2D";$2=$2+462216879;$3=$3+462216879;print $0}
        else if($1=="14"){$1="3A";$2=$2+454103970;$3=$3+454103970;print $0}
        else if($1=="16"){$1="3B";$2=$2+448155269;$3=$3+448155269;print $0}
        else if($1=="18"){$1="3D";$2=$2+476235359;$3=$3+476235359;print $0}
        else if($1=="20"){$1="4A";$2=$2+452555092;$3=$3+452555092;print $0}
        else if($1=="22"){$1="4B";$2=$2+451014251;$3=$3+451014251;print $0}
        else if($1=="24"){$1="4D";$2=$2+451004620;$3=$3+451004620;print $0}
        else if($1=="26"){$1="5A";$2=$2+453230519;$3=$3+453230519;print $0}
        else if($1=="28"){$1="5B";$2=$2+451372872;$3=$3+451372872;print $0}
        else if($1=="30"){$1="5D";$2=$2+451901030;$3=$3+451901030;print $0}
        else if($1=="32"){$1="6A";$2=$2+452440856;$3=$3+452440856;print $0}
        else if($1=="34"){$1="6B";$2=$2+452077197;$3=$3+452077197;print $0}
        else if($1=="36"){$1="6D";$2=$2+450509124;$3=$3+450509124;print $0}
        else if($1=="38"){$1="7A";$2=$2+450046986;$3=$3+450046986;print $0}
        else if($1=="40"){$1="7B";$2=$2+453822637;$3=$3+453822637;print $0}
        else if($1=="42"){$1="7D";$2=$2+453812268;$3=$3+453812268;print $0}

        }'  > ${infile}.1.txt


# 2 Alignment
## 2.1 long sequence
like reference genome

`nohup mashmap -r cs.1A.fa -q kg.1A.fa -t 10 -s 1000000 &`

I use this command to test the repeat sequence of iwgcs.v1.0, and:
```
#(212470367-27413157)/(223000000-37000000)*(185000000-37000000)+27413157
#(383127617-214581431)/(397000000-229000000)*(320000000-229000000)+214581431
```
these are the two regions of the genome that are aligned.

## 2.2 short sequence
There is an official script used to align short sequence data in LuLab and used in projects including Vmap3 and Vmap4.

2 steps:
- alignment
    - `bwa`
    - `bwa-mem2`: faster than `bwa`, only some line are less for a quality tag than `bwa`
- sort and filter: `samtools`

Vmap3 script:

In [ ]:
%%bash
for i in `cat fqlist.txt`
do

bwa mem -t 20 -R '@RG\tID:Aegilops\tPL:illumina\tSM:Aegilops' /data1/publicData/wheat/reference/v1.0/D/bwaLib/d_iwgscV1.fa.gz ${i}_1.fq.gz ${i}_2.fq.gz | samtools view -S -b -> ${i}.bam

samtools sort -n -m 4G -@ 20 -o ${i}.namesort.bam -O bam ${i}.bam \
 && samtools fixmate -@ 20 -m ${i}.namesort.bam ${i}.fixmate.bam \
 && samtools sort -m 4G -@ 20 -o ${i}.fixmate.pos.bam -O bam ${i}.fixmate.bam \
 && rm -f ${i}.namesort.bam \
 && samtools markdup -@ 20 -r ${i}.fixmate.pos.bam ${i}.rmdup.bam \
 && rm -f ${i}.fixmate.bam \
 && rm -f ${i}.fixmate.pos.bam

done


Vmap4 script: `bwa-mem2` version

In [ ]:
%%bash
#! /bin/bash

# file name: align-new
# the formal template for reads alignment in Lulab (used in Vmap4 Watkins)
## $1: number
## $2: /path/to/index
## $3: /path/to/fastq_data

mapfile -t fqlist < fqlist"$1".txt
mapfile -t smlist < smlist"$1".txt
size=${#fqlist[@]}
for (( i = 0; i < size; i++ ));
do
bwa-mem2 mem -t 20 -R "@RG\tID:Triticum\tPL:illumina\tSM:${smlist[i]}" "$2"/abd_iwgscV1.fa.gz "$3"/${fqlist[i]}/${fqlist[i]}_f1.fastq.gz "$3"/${fqlist[i]}/${fqlist[i]}_r2.fastq.gz | samtools view -S -b -> ${fqlist[i]}.bam
samtools sort -n -m 4G -@ 20 -o ${fqlist[i]}.namesort.bam -O bam ${fqlist[i]}.bam && samtools fixmate -@ 20 -m ${fqlist[i]}.namesort.bam ${fqlist[i]}.fixmate.bam && samtools sort -m 4G -@ 20 -o ${fqlist[i]}.fixmate.pos.bam -O bam ${fqlist[i]}.fixmate.bam && rm -f ${fqlist[i]}.namesort.bam && samtools markdup -@ 20 -r ${fqlist[i]}.fixmate.pos.bam ${fqlist[i]}.rmdup.bam && rm -f ${fqlist[i]}.fixmate.bam && rm -f ${fqlist[i]}.fixmate.pos.bam && rm -f ${fqlist[i]}.bam
done


Vmap4 script: `bwa` version

In [ ]:
%%bash

mapfile -t fqlist < fqlist.txt
mapfile -t smlist < smlist.txt
size=${#fqlist[@]}
for (( i = 0; i < size; i++ ));
do
bwa mem -t 20 -R "@RG\tID:Triticum\tPL:illumina\tSM:${smlist[i]}" /nfs/public_data/wheat/00_genome/reference/v1.0/ABD/bwaLib/abd_iwgscV1.fa.gz "$1"/${fqlist[i]}/${fqlist[i]}_f1.fastq.gz "$1"/${fqlist[i]}/${fqlist[i]}_r2.fastq.gz | samtools view -S -b -> ${fqlist[i]}.bam
samtools sort -n -m 4G -@ 20 -o ${fqlist[i]}.namesort.bam -O bam ${fqlist[i]}.bam && samtools fixmate -@ 20 -m ${fqlist[i]}.namesort.bam ${fqlist[i]}.fixmate.bam && samtools sort -m 4G -@ 20 -o ${fqlist[i]}.fixmate.pos.bam -O bam ${fqlist[i]}.fixmate.bam && rm -f ${fqlist[i]}.namesort.bam && samtools markdup -@ 20 -r ${fqlist[i]}.fixmate.pos.bam ${fqlist[i]}.rmdup.bam && rm -f ${fqlist[i]}.fixmate.bam && rm -f ${fqlist[i]}.fixmate.pos.bam
done


## 2.3 filter
View `.bam` file:
- `samtools view filename.bam | less -S`


